In [ ]:
!pip install transformers datasets accelerate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from datasets import Dataset

# Load your JSON file
df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/combined_training_data.json")

# If file is JSON Lines
# df = pd.read_json("/content/drive/MyDrive/Notebooks/ref_training_data.json", lines=True)

# Remove exact duplicates
df = df.drop_duplicates()

# Optionally, remove rows with duplicate text only (keep first occurrence)
# df = df.drop_duplicates(subset="text")

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Preview
print(f"Dataset length after duplicates removed: {len(dataset)}")
print(dataset[0])



Dataset length after duplicates removed: 621
{'text': 'Copyright 2014 American Medical Association. All rights reserved. Exercise and Depressive Symptoms in Adolescents A Longitudinal Cohort Study Umar Toseeb, PhD; Soren Brage, PhD; Kirsten Corder, PhD; Valerie J. Dunn, BEd; Peter B. Jones, MD, PhD, MRCPsych; Matthew Owens, PhD; Michelle C. St Clair, PhD; Esther M. F. van Sluijs, PhD; Ian M. Goodyer, MD, FRCPsych IMPORTANCE Physical activity (PA) may have a positive effect on depressed mood. However, whether it can act as a protective factor against developing depressive symptoms in adolescence is largely unknown. OBJECTIVE To investigate the association between objectively measured PA and depressive symptoms during 3 years of adolescence. DESIGN, SETTING, AND PARTICIPANTS We performed a longitudinal study between November 1, 2005, and January 31, 2010, of a community-based sample from Cambridgeshire and Suffolk, United Kingdom, that included 736 participants (mean [SD] age, 14.5 years

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# === 1. Label mapping ===
label_map = {"4*": 0, "3*": 1, "2*": 2, "1*": 3}
dataset = dataset.map(lambda x: {"label": label_map.get(x["label"], -1)})
dataset = dataset.filter(lambda x: x["label"] != -1)

# === 2. Load BigBird tokenizer ===
model_name = "google/bigbird-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# === 3. Tokenize dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=4096)

tokenized_dataset = dataset.map(tokenize, batched=True)

# === 4. Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# === 5. TrainingArguments
training_args = TrainingArguments(
    output_dir="./bigbird_ref_model",
    per_device_train_batch_size=1,              # bigbird is memory-heavy
    gradient_accumulation_steps=4,              # simulate batch size of 4
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=5,
    save_steps=10,
    save_total_limit=2,
    report_to="none"                            # avoid wandb, etc.
)

# === 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# === 7. Train!
trainer.train()

# === 8. Save the model
trainer.save_model("/content/bigbird_ref_star_classifier")
tokenizer.save_pretrained("/content/bigbird_ref_star_classifier")

print("✅ BigBird fine-tuning complete. Model saved to: /content/bigbird_ref_star_classifier")


Map:   0%|          | 0/621 [00:00<?, ? examples/s]

Filter:   0%|          | 0/621 [00:00<?, ? examples/s]

Map:   0%|          | 0/621 [00:00<?, ? examples/s]

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-53-2162506652.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
5,1.368200
10,1.300500
15,1.106000
20,0.954700
25,1.248000
30,0.895400
35,0.958000
40,1.199900
45,0.868600
50,0.998800


✅ BigBird fine-tuning complete. Model saved to: /content/bigbird_ref_star_classifier


**INFERENCE**

In [ ]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.5 MB/s eta 0:00:00


In [ ]:
import zipfile
import os

zip_path = "/content/new_pdfs.zip"
extract_dir = "/content/ref_pdfs"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Extracted to:", extract_dir)


✅ Extracted to: /content/ref_pdfs


In [ ]:
import os
import fitz  # PyMuPDF
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# === 1. Paths ===
pdf_folder = "/content/ref_pdfs/new_pdfs"  # 🔁 upload or mount this folder
model_path = "/content/bigbird_ref_star_classifier"
output_csv = "/content/ref_predictions.csv"

# === 2. Load model + tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# === 3. Label mapping ===
label_map = {0: "4*", 1: "3*", 2: "2*", 3: "1*"}

# === 4. PDF text extractor ===
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = " ".join([page.get_text() for page in doc])
        return " ".join(text.strip().split())  # clean up
    except Exception as e:
        print(f"❌ Error in {pdf_path}: {e}")
        return None

# === 5. Predict from text ===
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=4096)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred].item()
        return label_map[pred], round(confidence, 4)

# === 6. Process PDFs ===
results = []
print(f"📂 Scanning folder: {pdf_folder}")

for fname in os.listdir(pdf_folder):
    if fname.endswith(".pdf"):
        fpath = os.path.join(pdf_folder, fname)
        print(f"\n🔍 Processing: {fname}")

        text = extract_text_from_pdf(fpath)
        if text:
            label, conf = predict(text)
            print(f"✅ Predicted: {label} (Confidence: {conf})")
            results.append({"filename": fname, "predicted_label": label, "confidence": conf})
        else:
            print("⚠️ Skipped due to extraction failure.")

# === 7. Save to CSV ===
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)
print(f"\n📄 All predictions saved to: {output_csv}")


📂 Scanning folder: /content/ref_pdfs/new_pdfs

🔍 Processing: 10.1080_1357650X.2018.1461897.pdf
✅ Predicted: 3* (Confidence: 0.4907)

🔍 Processing: 10.1037_a0039161.pdf
✅ Predicted: 3* (Confidence: 0.6993)

🔍 Processing: 10.7554_elife.33323.pdf
✅ Predicted: 4* (Confidence: 0.6442)

🔍 Processing: 10.1038_s41380-018-0118-1.pdf
✅ Predicted: 4* (Confidence: 0.5171)

🔍 Processing: 10.1136_bmj.i1030.pdf
✅ Predicted: 3* (Confidence: 0.7108)

🔍 Processing: torok-et-al-2019-rationality-in-joint-action-maximizing-coefficiency-in-coordination.pdf
✅ Predicted: 3* (Confidence: 0.6882)

📄 All predictions saved to: /content/ref_predictions.csv
